<h4> This code is part of FLAUTO. It implements FedHPO. Date: 01/09/2025 </h4>
<h4> Contact: rakibul.haque@utsa.edu </h4>  
<h4> Cite as: R. U. Haque and P. Markopoulos,"Federated Learning with Automated Dual-Level Hyperparameter Tuning", 2025 <h4>

<H1>Import Libraries</H1>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torchvision.datasets import CIFAR10
from collections import Counter
import random
import copy
import torch.nn.functional as F
import os
import sys
import time
import pickle
import pandas as pd
from torch import Tensor
from typing import Type
import torchvision.models as models
from torchvision import transforms
from copy import deepcopy

# Set font family for plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

<H1>Load Dataset</H1>

In [ ]:
subset_dataset = torch.load('val_dataset.pth',weights_only=False)
remaining_dataset = torch.load('test_dataset.pth',weights_only=False)

# Create DataLoaders
val_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(remaining_dataset, batch_size=32, shuffle=False)

<H1>IID</H1>

In [ ]:
def distribute_dataset_equally(dataset, num_clients):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    # Distribute data
    client_data = [[] for _ in range(num_clients)]
    for label, data in class_data.items():
        data_len = len(data)
        base_size = data_len // num_clients
        remain = data_len - base_size * num_clients

        current_idx = 0
        for i in range(num_clients):
            end_idx = current_idx + base_size + (1 if i < remain else 0)
            client_data[i].extend(data[current_idx:end_idx])
            current_idx = end_idx


    print_iid_distribution(client_data)
    plot_iid_dataset(client_data,num_clients)

    return client_data

In [ ]:
#print client iid_data distribution
def print_iid_distribution(iid_datasets):
    # Check if the distribution is correct
    for i, client_data in enumerate(iid_datasets):
        print(f"Client {i + 1} data size: {len(client_data)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [ ]:
# Plot client iid_data distribution
def plot_iid_dataset(iid_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in iid_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define colors for each class
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")
    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number',labelpad=5)
    ax.set_ylabel('CIFAR-10 Classes',labelpad=15)
    ax.set_zlabel('Fraction of Distribution',labelpad=5)
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], rotation=45)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names, rotation=-60)

    #ax.view_init(elev=40, azim=60)
    ax.view_init(elev=40, azim=10)

    plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99)

    plt.show()

<H1>non-IID</H1>

In [ ]:
#Print the distribution non IID
def print_distribution(client_data):
    # Check distribution
    for l, client_data_value in enumerate(client_data):
        print(f"Client {l + 1} data size: {len(client_data_value)}")
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data_value:
            class_counts[label] += 1
        print(f"Class distribution: {class_counts}")

In [ ]:
# #Non IID code
def distribute_dataset_dirichlet(dataset, num_clients, alpha):
    # Group data by class
    class_data = {}
    for data, label in dataset:
        if label not in class_data:
            class_data[label] = []
        class_data[label].append((data, label))

    client_data = [[] for _ in range(num_clients)]
    for _, data_list in class_data.items():
        # Shuffle data for randomness
        np.random.shuffle(data_list)

        # Get proportions for data split based on Dirichlet distribution
        proportions = np.random.dirichlet([alpha]*num_clients)
        # print("Proportions: ", proportions)
        total_data = len(data_list)
        # print("total_data: ", total_data)
        data_splits = [int(proportions[i]*total_data) for i in range(num_clients)]
        # print("Data_Split: ", data_splits)

        # Adjust the splits to account for rounding errors
        # print("Before data_split:",data_splits)
        data_splits[-1] += total_data - sum(data_splits)
        print("After data_split:",data_splits)

        start_idx = 0
        for i, split in enumerate(data_splits):
            end_idx = start_idx + split
            client_data[i].extend(data_list[start_idx:end_idx])
            start_idx = end_idx
    print("Client Number: ",num_clients, "Alpha: ", alpha)
    print_distribution(client_data)
    plot_distribution(client_data, num_clients )
    return client_data

In [ ]:
#show the graphs for nonIID
def plot_distribution(client_datasets, num_clients):
    # CIFAR-10 class names
    class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # Calculate fraction of distribution for each class across clients
    fractions = []
    for client_data in client_datasets:
        class_counts = {j: 0 for j in range(10)}
        for _, label in client_data:
            class_counts[label] += 1
        total_data = len(client_data)
        # print(total_data)
        fractions.append([class_counts[i] / total_data for i in range(10)])
    fractions = np.array(fractions)
    # Define distinct colors for classes using a colormap
    colors = ["r", "g", "b", "c", "m", "y", "#FFA500", "#FF00FF", "#808080", "#00FF00"]
    
    # Generate the 3D bar chart
    fig = plt.figure(figsize=(10, 7))  # Adjust the size here
    ax = fig.add_subplot(111, projection='3d')

    xpos, ypos = np.meshgrid(np.arange(num_clients), np.arange(10), indexing="ij")

    for i in range(10):
        ax.bar3d(xpos[:, i], ypos[:, i], np.zeros_like(xpos[:, i]),0.75, 0.75, fractions[:, i],shade=True, color=colors[i])

    ax.set_xlabel('Client Number')
    ax.set_ylabel('CIFAR-10 Classes')
    ax.set_zlabel('Fraction of Distribution')
    ax.set_title('Distribution of CIFAR-10 Classes across Clients based on Dirichlet Distribution')
    ax.set_xticks(np.arange(0.5, num_clients))
    ax.set_xticklabels([str(i+1) for i in range(num_clients)], fontsize=12)
    ax.set_yticks(np.arange(0.5, 10))
    ax.set_yticklabels(class_names)
    plt.tight_layout()
    plt.show()

# Measures

In [ ]:
def accuracy(outp, target):
    """Computes accuracy"""
    with torch.no_grad():
        pred = torch.argmax(outp, dim=1)
        correct = pred.eq(target).float().sum().item()
        return 100.0 * correct / target.size(0)

In [ ]:
def Print(string, dictionary):
    first_key = next(iter(dictionary))
    first_value = dictionary[first_key]
    print(f"{string}:{first_key}: {first_value[0][0]}\n")

In [ ]:
def forbinus_norm_function(w_i):
    value = 0
    for k in w_i.keys():
        value += torch.linalg.norm(w_i[k])
    return value.item()

In [ ]:
def model_deviation_function(w_i, w_f):
    model_deviation = 0
    for k in w_i.keys():
        model_deviation += torch.linalg.norm(w_f[k].to(torch.float) - w_i[k].to(torch.float)) / torch.linalg.norm(w_i[k].to(torch.float))
    #print(model_deviation.item())
    return model_deviation.item()

# Model

In [ ]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # Change output to 10 for CIFAR-10 classes

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.relu5(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

# Train

In [ ]:
def train(i_weights, epochs, train_loader, le_rate, cli,roun, epoch_flag):
    global opti

    e_a, e_l=test(i_weights, test_loader)
    test_loss_epoch.append(e_l)
    
    local_model = model().to(device)
    criterion = nn.CrossEntropyLoss()
    if opti=="adam":
        optimizer = torch.optim.Adam(local_model.parameters(), lr=le_rate,  weight_decay=0.004)
    elif opti=="sgd":
        optimizer = torch.optim.SGD(local_model.parameters(), lr=le_rate,  weight_decay=0.004)
    
    epoch_train_accuracy=0 
    epoch_train_loss=0
    epoch_test_accuracy=0
    epoch_test_loss=0
    epoch_rmd=0

    local_model.load_state_dict(i_weights)

    local_model.train()  # Set the model to training mode

    flag_counter=0
    lr=le_rate
    # initial weights cathing and printing
    initial_weights = {k: v.clone() for k, v in local_model.state_dict().items()}
    #Print("Model's inside the function Initial weights for client",initial_weights)

    # Training loop
    for epoch in range(epochs):
        epoch_flag=epoch_flag+1
        # gradients_this_epoch = {}
        total_samples = 0
        total_loss=0
        correct_samples = 0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward + backward + optimize
            outputs = local_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()

            _, predicted = outputs.max(1)  # Get the index of the maximum value in outputs (predicted class)
            total_samples += labels.size(0)
            correct_samples += predicted.eq(labels).sum().item()
        
        if(total_samples!=0 and len(train_loader)!=0):
            epoch_accuracy = 100 * correct_samples / total_samples
            epoch_loss = total_loss / len(train_loader)
        else:
            epoch_accuracy = 100 * correct_samples / (total_samples+1)
            epoch_loss = total_loss / (len(train_loader)+1)
        print(f"Round {roun}, cleint {cli+1}, epoch {epoch+1}, current LR: {optimizer.param_groups[0]['lr']}, epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} ")

        a_f_w = {k: v.clone() for k, v in local_model.state_dict().items()}
        
        e_a, e_l=test(a_f_w, test_loader)
        test_loss_epoch.append(e_l)
        
        print(f"test loss difference {test_loss_epoch[epoch] - test_loss_epoch[epoch-1]}")
        
        if abs(test_loss_epoch[epoch] - test_loss_epoch[epoch-1]) < 0.002:
            lr= lr * 0.5
            optimizer.param_groups[0]['lr'] = lr

            current_learning_rate = optimizer.param_groups[0]['lr']
            #print("Current learning rate:", current_learning_rate)

            flag_counter=flag_counter+1

            if flag_counter==5:
                break


    
    f_weights = {k: v.clone() for k, v in local_model.state_dict().items()}

    #print(f"\n Round {roun}, cleint {cli}: epoch_accuracy {epoch_accuracy}, epoch_loss {epoch_loss} \n")
    epoch_train_accuracy=epoch_accuracy
    epoch_train_loss=epoch_loss
    epoch_test_accuracy, epoch_test_loss= test(f_weights, test_loader)
    
    
    epoch_rmd=model_deviation_function(initial_weights,f_weights)
    
    #saving data into dataframe
    epoch_data = [epoch_train_accuracy, epoch_train_loss, epoch_test_accuracy, epoch_test_loss, epoch_rmd]
    epoch_results.loc[len(epoch_results)] = epoch_data
    
    model_update = {}
    for key in local_model.state_dict():
        model_update[key] = torch.sub(i_weights[key], f_weights[key])
    
    return epoch_accuracy,epoch_loss, epoch_flag, model_update

# Test

In [ ]:
def test(w,data):
    lmodel = model().to(device)
    criterion = nn.CrossEntropyLoss()  # Assuming a classification task
    #optimizer = torch.optim.SGD(lmodel.parameters(), lr=learning_rate)
    lmodel.load_state_dict(w)
    lmodel.eval()

    #checking the weights
    tw = lmodel.state_dict()
    #Print("Model's before testing the weights in global model",tw)

    # Evaluation phase for test set
    acc_list = []
    loss_list = []

    with torch.no_grad():
        for j, data in enumerate(data, 0):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            out = lmodel(images)
            # Calculate loss
            loss = criterion(out, labels)
            loss_list.append(loss.item())
            #calculate accuracy
            acc = accuracy(out, labels)
            acc_list.append(acc)
    test_loss = np.mean(loss_list)
    test_accuracy = np.mean(acc_list)
    #print("Model's Test accuracy : {:.2f}%".format(test_accuracy))
    return test_accuracy, test_loss

# FL Structure

In [ ]:
def average_updates(w, n_k):
    w_avg = deepcopy(w[0])
    for key in w_avg.keys():
        w_avg[key] = torch.mul(w_avg[key], n_k[0])
        for i in range(1, len(w)):
            w_avg[key] = torch.add(w_avg[key], w[i][key], alpha=n_k[i])
        w_avg[key] = torch.div(w_avg[key], sum(n_k))
    return w_avg

In [ ]:
def federated_learning(i_w, data_client, C, P, R, E, learning_rate, b_size):
    
    global total_cleints_list, participating_client_list, v
    
    global_model.load_state_dict(i_w)
    #Print("Model's initial weights", i_w)

    #loop for round
    for r in range(1,R+1):
        round_train_accuracy=0
        round_train_loss=0
        round_test_accuracy=0
        round_test_loss=0
        epoch_flag=0

        #saving initial weights for spiking model
        i_w = {k: v.clone() for k, v in global_model.state_dict().items()}
        #Print("Model's initial weights", i_w)
        
        #colleting weights and results
        data_size=[]
        all_final_weights={}
        all_cleints_updates=[]
        train_accuracy_list=[]
        train_loss_list=[]
        
        # Randomly select clients
        selected_clients = random.sample(total_cleints_list, P)
        participating_client_list.append(selected_clients)

        #loop for client
        for c, data in enumerate(data_client):
            
            if(c in selected_clients):
                
                train_loader = torch.utils.data.DataLoader(data, batch_size=b_size, shuffle=True, drop_last=True)
                
                #train model
                train_accuracy, train_loss, epoch_flag, model_update = train(i_w, E, train_loader, learning_rate, c, r,epoch_flag)

                train_accuracy_list.append(train_accuracy)
                train_loss_list.append(train_loss)
                
                all_cleints_updates.append(model_update)

                data_size.append(len(train_loader))

            else:
                print(f"client {c} is not selectecd")
        
        round_epoch=(epoch_flag)
        #print("Total number of selected clients is", client_counter)
        round_train_loss=sum(train_loss_list)/len(train_loss_list)
        round_train_accuracy=sum(train_accuracy_list)/len(train_accuracy_list)
        print(f"Model's Round: {r}, train accuracy of model: {round_train_accuracy}, train loss of model: {round_train_loss} \n\n")

        update_avg = average_updates( all_cleints_updates, data_size)

        if v is None:
            v = deepcopy(update_avg)
        else:
            for key in v.keys():
                v[key] = update_avg[key] + (v[key] * 0.99)
                
        for key in i_w:
            all_final_weights[key]=i_w[key] - (v[key] * 1)

        round_test_accuracy, round_test_loss=test(all_final_weights, test_loader)
        print(f"Model's Round: {r}, test accuracy of model: {round_test_accuracy}, test loss of model: {round_test_loss} \n\n")

        #model deviation code
        round_rmd=model_deviation_function(i_w, all_final_weights)
        #print("Model deviation values: ", model_deviation)

        #saving data into dataframe
        round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch]
        round_results.loc[len(round_results)] = round_data
            
        global_model.load_state_dict(all_final_weights)
        print("round", r, "completed")

# Main Function

# Define parameters

In [ ]:
#===========================Parameters==============================================================
client_no=20
participating_client=20
epochs=5
learning_rate=0.01
round_no=30
batch_size=128
distributions = "non_iid" # 'non_iid'
data_class=10
alpha=0.5
opti="sgd" # or sgd or adam
v=None
# List of clients

test_loss_epoch=[]

total_cleints_list = list(range(0, client_no))
# print(total_cleints_list)
participating_client_list=[]

method="fed_HPO"

# Define dataframe for round results
round_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd', 'epoch']
round_results = pd.DataFrame(columns=round_columns)

# Define dataframe for epoch results
epoch_columns = ['train_accuracy', 'train_loss', 'test_accuracy', 'test_loss', 'rmd']
epoch_results = pd.DataFrame(columns=epoch_columns)

#===================================loading the saved weight list====================================================
global_model = model().to(device)
# initial_weights={k: v.clone() for k, v in global_model.state_dict().items()}
# Save the initial weights
file_path = "s_cnn.pth"
# torch.save(initial_weights, file_path)
initial_weights=torch.load(file_path,weights_only=True)
Print("Model's initial weights", initial_weights)


<H1>Divide data among cleints</H1>

In [ ]:
# Load client_datasets from a file
if distributions == 'iid':
    with open('20_client_datasets_IID.pkl', 'rb') as f:
        client_datasets = pickle.load(f)

elif distributions == 'non_iid' and alpha==0.5:
    with open('20_client_datasets_non_IID_0_5.pkl', 'rb') as f:
        client_datasets = pickle.load(f)
    
elif distributions == 'non_iid' and alpha==0.125:
    with open('20_client_datasets_non_IID_0_125.pkl', 'rb') as f:
        client_datasets = pickle.load(f)
        
print("client_datasets loaded successfully.")
print_distribution(client_datasets)

<H1>Round zero</H1>

In [ ]:
#train accuracy for cleints
round_train_accuracy=0
round_train_loss=0

train_accuracy_list=[]
train_loss_list=[]
for c, data in enumerate(client_datasets):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
    train_accuracy, train_loss=test(initial_weights, train_loader)
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)
round_train_accuracy=(sum(train_accuracy_list)/len(train_accuracy_list))
round_train_loss=(sum(train_loss_list)/len(train_loss_list))


#test accuracy for server
round_test_accuracy=0
round_test_loss=0
test_accuracy,test_loss=test(initial_weights,test_loader)
round_test_accuracy=(test_accuracy)
round_test_loss=(test_loss)

round_rmd=0
round_epoch=0

round_data = [round_train_accuracy, round_train_loss, round_test_accuracy, round_test_loss, round_rmd, round_epoch]
round_results.loc[len(round_results)] = round_data

Print("initial_weights", initial_weights)
print(f' train accuracy: {round_train_accuracy}\n train_loss: {round_train_loss}\n test_accuracy: {round_test_accuracy}\n test_loss: {round_test_loss}')

<H1>Run FL</H1>

In [ ]:
federated_learning(initial_weights, client_datasets, client_no, participating_client, round_no, epochs, learning_rate, batch_size)

test loss difference 0.22199484791618573
Round 9, cleint 1, epoch 5, current LR: 0.01, epoch_accuracy 88.24404761904762, epoch_loss 0.3249958817447935 
test loss difference 0.26447960286856453
Round 9, cleint 1, epoch 6, current LR: 0.01, epoch_accuracy 89.28571428571429, epoch_loss 0.2850254745710464 
test loss difference 0.15022736387892666
Round 9, cleint 1, epoch 7, current LR: 0.01, epoch_accuracy 91.36904761904762, epoch_loss 0.23963311917725064 
test loss difference 0.15087825117019849
Round 9, cleint 1, epoch 8, current LR: 0.01, epoch_accuracy 88.24404761904762, epoch_loss 0.2698503250167483 
test loss difference -0.006767111464430542
Round 9, cleint 1, epoch 9, current LR: 0.01, epoch_accuracy 91.96428571428571, epoch_loss 0.2145306876017934 
test loss difference -0.07493197270475616
Round 9, cleint 1, epoch 10, current LR: 0.01, epoch_accuracy 92.85714285714286, epoch_loss 0.1929805138636203 
test loss difference -0.03232282114485985
Round 9, cleint 2, epoch 1, current LR: 0

test loss difference -0.006767111464430542
Round 9, cleint 6, epoch 9, current LR: 0.01, epoch_accuracy 94.39102564102564, epoch_loss 0.15842558933883652 
test loss difference -0.07493197270475616
Round 9, cleint 6, epoch 10, current LR: 0.01, epoch_accuracy 95.21901709401709, epoch_loss 0.1414154572173571 
test loss difference -0.03232282114485985
Round 9, cleint 7, epoch 1, current LR: 0.01, epoch_accuracy 66.50280898876404, epoch_loss 0.94037785958708 
test loss difference 0.034754858230249486
Round 9, cleint 7, epoch 2, current LR: 0.01, epoch_accuracy 72.78792134831461, epoch_loss 0.7170638552542483 
test loss difference 0.014864399029423936
Round 9, cleint 7, epoch 3, current LR: 0.01, epoch_accuracy 75.66713483146067, epoch_loss 0.6452355877067266 
test loss difference 0.9918816912288482
Round 9, cleint 7, epoch 4, current LR: 0.01, epoch_accuracy 78.19522471910112, epoch_loss 0.5958592061246379 
test loss difference 0.22199484791618573
Round 9, cleint 7, epoch 5, current LR: 0.

Round 10, cleint 2, epoch 1, current LR: 0.01, epoch_accuracy 85.78789893617021, epoch_loss 0.4185512318335315 
test loss difference -0.5587290772995628
Round 10, cleint 2, epoch 2, current LR: 0.01, epoch_accuracy 89.22872340425532, epoch_loss 0.29116891772347564 
test loss difference 0.014864399029423936
Round 10, cleint 2, epoch 3, current LR: 0.01, epoch_accuracy 90.85771276595744, epoch_loss 0.2538822967796884 
test loss difference 0.9918816912288482
Round 10, cleint 2, epoch 4, current LR: 0.01, epoch_accuracy 91.95478723404256, epoch_loss 0.2211886317013426 
test loss difference 0.22199484791618573
Round 10, cleint 2, epoch 5, current LR: 0.01, epoch_accuracy 92.56981382978724, epoch_loss 0.20309217995468606 
test loss difference 0.26447960286856453
Round 10, cleint 2, epoch 6, current LR: 0.01, epoch_accuracy 92.73603723404256, epoch_loss 0.1883007553306983 
test loss difference 0.15022736387892666
Round 10, cleint 2, epoch 7, current LR: 0.01, epoch_accuracy 93.43417553191489,

test loss difference 0.22199484791618573
Round 10, cleint 7, epoch 5, current LR: 0.01, epoch_accuracy 78.54634831460675, epoch_loss 0.5508563595541408 
test loss difference 0.26447960286856453
Round 10, cleint 7, epoch 6, current LR: 0.01, epoch_accuracy 80.72331460674157, epoch_loss 0.5084389637360412 
test loss difference 0.15022736387892666
Round 10, cleint 7, epoch 7, current LR: 0.01, epoch_accuracy 81.46067415730337, epoch_loss 0.4963640565617701 
test loss difference 0.15087825117019849
Round 10, cleint 7, epoch 8, current LR: 0.01, epoch_accuracy 82.37359550561797, epoch_loss 0.47042512826705246 
test loss difference -0.006767111464430542
Round 10, cleint 7, epoch 9, current LR: 0.01, epoch_accuracy 84.05898876404494, epoch_loss 0.4275606954365634 
test loss difference -0.07493197270475616
Round 10, cleint 7, epoch 10, current LR: 0.01, epoch_accuracy 84.90168539325843, epoch_loss 0.39823285498645866 
test loss difference -0.03232282114485985
Round 10, cleint 8, epoch 1, curre

test loss difference 0.15022736387892666
Round 11, cleint 2, epoch 7, current LR: 0.01, epoch_accuracy 93.25132978723404, epoch_loss 0.1829522377910449 
test loss difference 0.15087825117019849
Round 11, cleint 2, epoch 8, current LR: 0.01, epoch_accuracy 94.33178191489361, epoch_loss 0.16228476261838953 
test loss difference -0.006767111464430542
Round 11, cleint 2, epoch 9, current LR: 0.01, epoch_accuracy 94.31515957446808, epoch_loss 0.14883946048769545 
test loss difference -0.07493197270475616
Round 11, cleint 2, epoch 10, current LR: 0.01, epoch_accuracy 95.01329787234043, epoch_loss 0.14187757502448686 
test loss difference -0.03232282114485985
Round 11, cleint 3, epoch 1, current LR: 0.01, epoch_accuracy 67.6875, epoch_loss 0.7457880248626073 
test loss difference -0.47828605723457196
Round 11, cleint 3, epoch 2, current LR: 0.01, epoch_accuracy 72.5625, epoch_loss 0.6317247583468755 
test loss difference 0.014864399029423936
Round 11, cleint 3, epoch 3, current LR: 0.01, epoc

...

test loss difference 0.15022736387892666
Round 98, cleint 7, epoch 7, current LR: 0.01, epoch_accuracy 84.48033707865169, epoch_loss 0.4260154292824563 
test loss difference 0.15087825117019849
Round 98, cleint 7, epoch 8, current LR: 0.01, epoch_accuracy 85.49859550561797, epoch_loss 0.3969577429502198 
test loss difference -0.006767111464430542
Round 98, cleint 7, epoch 9, current LR: 0.01, epoch_accuracy 85.42837078651685, epoch_loss 0.39273962177587357 
test loss difference -0.07493197270475616
Round 98, cleint 7, epoch 10, current LR: 0.01, epoch_accuracy 86.58707865168539, epoch_loss 0.37113855787542427 
test loss difference -0.03232282114485985
Round 98, cleint 8, epoch 1, current LR: 0.01, epoch_accuracy 90.13157894736842, epoch_loss 0.30712088950370486 
test loss difference -0.011508372264167566
Round 98, cleint 8, epoch 2, current LR: 0.01, epoch_accuracy 92.10526315789474, epoch_loss 0.22346662375095644 
test loss difference 0.014864399029423936
Round 98, cleint 8, epoch 3, 

test loss difference -0.006767111464430542
Round 99, cleint 2, epoch 9, current LR: 0.01, epoch_accuracy 94.94680851063829, epoch_loss 0.14236803252765157 
test loss difference -0.07493197270475616
Round 99, cleint 2, epoch 10, current LR: 0.01, epoch_accuracy 95.3125, epoch_loss 0.1358344869569261 
test loss difference -0.03232282114485985
Round 99, cleint 3, epoch 1, current LR: 0.01, epoch_accuracy 71.58333333333333, epoch_loss 0.6591716978947322 
test loss difference 0.4912197231865538
Round 99, cleint 3, epoch 2, current LR: 0.01, epoch_accuracy 76.375, epoch_loss 0.5554342416922251 
test loss difference 0.014864399029423936
Round 99, cleint 3, epoch 3, current LR: 0.01, epoch_accuracy 78.125, epoch_loss 0.5242874884605407 
test loss difference 0.9918816912288482
Round 99, cleint 3, epoch 4, current LR: 0.01, epoch_accuracy 78.91666666666667, epoch_loss 0.49623119294643403 
test loss difference 0.22199484791618573
Round 99, cleint 3, epoch 5, current LR: 0.01, epoch_accuracy 79.62

test loss difference 0.014864399029423936
Round 99, cleint 8, epoch 3, current LR: 0.01, epoch_accuracy 94.04605263157895, epoch_loss 0.17769094478142888 
test loss difference 0.9918816912288482
Round 99, cleint 8, epoch 4, current LR: 0.01, epoch_accuracy 93.71710526315789, epoch_loss 0.18053121472659864 
test loss difference 0.22199484791618573
Round 99, cleint 8, epoch 5, current LR: 0.01, epoch_accuracy 94.63815789473684, epoch_loss 0.15569812249588338 
test loss difference 0.26447960286856453
Round 99, cleint 8, epoch 6, current LR: 0.01, epoch_accuracy 94.86842105263158, epoch_loss 0.1474798304077826 
test loss difference 0.15022736387892666
Round 99, cleint 8, epoch 7, current LR: 0.01, epoch_accuracy 95.39473684210526, epoch_loss 0.13356909528374672 
test loss difference 0.15087825117019849
Round 99, cleint 8, epoch 8, current LR: 0.01, epoch_accuracy 95.49342105263158, epoch_loss 0.12748384432573068 
test loss difference -0.006767111464430542
Round 99, cleint 8, epoch 9, curre

test loss difference 0.22199484791618573
Round 100, cleint 3, epoch 5, current LR: 0.01, epoch_accuracy 79.10416666666667, epoch_loss 0.4854635359843572 
test loss difference 0.26447960286856453
Round 100, cleint 3, epoch 6, current LR: 0.01, epoch_accuracy 81.25, epoch_loss 0.45400695820649467 
test loss difference 0.15022736387892666
Round 100, cleint 3, epoch 7, current LR: 0.01, epoch_accuracy 81.72916666666667, epoch_loss 0.44498072465260824 
test loss difference 0.15087825117019849
Round 100, cleint 3, epoch 8, current LR: 0.01, epoch_accuracy 82.10416666666667, epoch_loss 0.4307288187742233 
test loss difference -0.006767111464430542
Round 100, cleint 3, epoch 9, current LR: 0.01, epoch_accuracy 82.47916666666667, epoch_loss 0.41452979187170663 
test loss difference -0.07493197270475616
Round 100, cleint 3, epoch 10, current LR: 0.01, epoch_accuracy 82.58333333333333, epoch_loss 0.41096984962622324 
test loss difference -0.03232282114485985
Round 100, cleint 4, epoch 1, current 

test loss difference -0.006767111464430542
Round 100, cleint 8, epoch 9, current LR: 0.01, epoch_accuracy 95.36184210526316, epoch_loss 0.12499371576858194 
test loss difference -0.07493197270475616
Round 100, cleint 8, epoch 10, current LR: 0.01, epoch_accuracy 96.34868421052632, epoch_loss 0.11583102599864728 
test loss difference -0.03232282114485985
Round 100, cleint 9, epoch 1, current LR: 0.01, epoch_accuracy 86.62476503759399, epoch_loss 0.36569873016691745 
test loss difference 0.13918892986858244
Round 100, cleint 9, epoch 2, current LR: 0.01, epoch_accuracy 88.36936090225564, epoch_loss 0.3184500587824032 
test loss difference 0.014864399029423936
Round 100, cleint 9, epoch 3, current LR: 0.01, epoch_accuracy 89.49130639097744, epoch_loss 0.2933667736422074 
test loss difference 0.9918816912288482
Round 100, cleint 9, epoch 4, current LR: 0.01, epoch_accuracy 90.26080827067669, epoch_loss 0.2723242758766265 
test loss difference 0.22199484791618573
Round 100, cleint 9, epoch 

# Reuslts

In [ ]:
# Define the folder and file name
folder_name = f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"  # Folder where the Excel file will be saved
file_name = "round_results.xlsx"


# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

round_results.to_excel(file_path, index=False)

print("DataFrame successfully written for round results.")

In [ ]:
# Define the folder and file name
folder_name =  f"{method}_{opti}_{learning_rate}_{participating_client}_{client_no}_{distributions}_{alpha}"   # Folder where the Excel file will be saved
file_name = "epoch_results.xlsx"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Full path where the Excel file will be saved
file_path = os.path.join(folder_name, file_name)

epoch_results.to_excel(file_path, index=False)

print("DataFrame successfully written for epoch results.")